In [1]:
%load_ext autoreload
%autoreload 2

# Analyse ruggedness for different datasets

In [17]:
from sklearn.manifold import TSNE
import itertools
from functools import partial
import json
import jax.numpy as jnp
import numpy as np
import os
import pandas as pd
import jax
import seaborn as sns
import matplotlib.pyplot as plt
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict, write_json
from synbio_morpher.utils.results.analytics.naming import get_true_interaction_cols
from evoscaper.scripts.init_from_hpos import init_from_hpos


jax.config.update('jax_platform_name', 'cpu')

jax.devices()

[cuda(id=1)]

# Biological resim: dataset 0211

## Add ruggedness to og dataset

In [ ]:
fn_ds0211 = 'data/simulate_circuits/2025_02_11__13_56_51/tabulated_mutation_info.json'
dir_ds0211_rug1 = 'data/07_ruggedness/2025_03_05__15_19_07'
dir_ds0211_rug2 = 'data/07_ruggedness/2025_03_06__22_11_21'

data = pd.read_json(fn_ds0211)
analytics1 = load_json_as_dict(os.path.join(dir_ds0211_rug1, 'analytics.json'))
analytics2 = load_json_as_dict(os.path.join(dir_ds0211_rug2, 'analytics.json'))
config = load_json_as_dict(os.path.join(dir_ds0211_rug1, 'config.json'))

In [14]:
analytics = {}
for k in analytics1:
    if k in analytics2:
        analytics[k] = np.concatenate([analytics1[k], analytics2[k]])
    else: print(k)

In [20]:
hpos = pd.Series(load_json_as_dict(os.path.join(os.path.dirname(config['fn_saves']), 'hpos_all.json')))
(
    rng, rng_model, rng_dataset,
    config_norm_x, config_norm_y, config_filter, config_optimisation, config_dataset, config_training, config_model,
    data, x_cols, df,
    x, cond, y, x_train, cond_train, y_train, x_val, cond_val, y_val,
    total_ds, n_batches, BATCH_SIZE, x_datanormaliser, x_methods_preprocessing, y_datanormaliser, y_methods_preprocessing,
    params, encoder, decoder, model, h2mu, h2logvar, reparam
) = init_from_hpos(hpos)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [21]:
data_og = pd.read_json(fn_ds0211)
len(data), len(data_og)

(2579790, 2579790)

In [ ]:
species = data['sample_name'].dropna().unique()

def add_rugg(analytics: dict, analytic: str):
    
    n_samples = interactions.shape[0]
    n_interactions = interactions.shape[1]
    n_perturbs = n_interactions + resimulate_analytics
    
    analytic_perturbed = jnp.array(
        analytics_perturbed[analytic]).reshape(n_samples, n_perturbs, -1)
    if resimulate_analytics:
        analytic_perturbed = analytic_perturbed[:, :-1, :]
        analytic_og = analytic_perturbed[:, -1, :]
    else:
        analytic_og = np.array(analytics_original[analytic][:n_samples])
        
    # If loaded from previous data where not all analytics were saved
    if analytic_perturbed.shape[-1] != analytic_og.shape[-1]: 
        analytic_perturbed = analytic_perturbed[..., -analytic_og.shape[-1]:]

    ruggedness = jax.vmap(partial(calculate_ruggedness_from_perturbations, eps=eps))(
        analytic_perturbed, analytic_og[:, None, :])

ruggedness1 = np.load(os.path.join(dir_ds0211_rug1, 'ruggedness.npy'))
ruggedness2 = np.load(os.path.join(dir_ds0211_rug2, 'ruggedness.npy'))
ruggedness = np.concatenate([ruggedness1, ruggedness2])
ruggedness = ruggedness[..., -len(species):]
data['ruggedness']

In [10]:
ruggedness[0], ruggedness.flatten()[:4]

(array([0.        , 0.4044732 , 0.40420768], dtype=float32),
 array([0.        , 0.4044732 , 0.40420768, 0.        ], dtype=float32))